# Find a nominal parametrization of the linear feedback control (fast turnover)


In [38]:
import numpy as np
import sing_birth_fus_utils_ap as utls
from mitonetworks.utls import reset_plots

%reload_ext autoreload
%autoreload 2

Get a parametrization motivated by literature

In [39]:
average_rate_fusion_per_mito = 0.023 # per min, Cagalinec13
beta = average_rate_fusion_per_mito * 24 * 60 # per day
nsp = 250 # "copy number per species"
gamma = beta * 2.0 * nsp/(7*nsp*nsp)
beta, gamma

(33.120000000000005, 0.03785142857142858)

In [40]:
mu= np.log(2)/2.0 

In [41]:
n_target = 1000.0

In [42]:
xi = 0.00
delta = 1.0

For a range of $b$ find the $\kappa$ which has a copy number of `n_target` and compute the CV

In [43]:
b_space = np.logspace(np.log10(1e-5),np.log10(1e-1),1001)

In [44]:
b_kappa_arr = []
kappa_space = n_target*10**np.linspace(-1,3,10000)

minimum_residuals = []

for i, b in enumerate(b_space):
    if i%100==0: print(i)    
    ms = 250.0 # arbitrary when delta = 1.0
    res = utls.residual_n_manual(n_target, ms, xi, gamma, beta, kappa_space, b, mu, delta,n_target)
    if np.min(res) < 2: # if the minimum residual is acceptable, i.e. we found a good kappa   
        kappa_opt = kappa_space[np.argmin(res)]
        b_kappa_arr.append([b, kappa_opt])
    else:
        minimum_residuals.append(np.min(res))
b_kappa_arr = np.array(b_kappa_arr)

0
100
200
300
400
500
600
700
800
900
1000


In [45]:
b_kappa_arr

array([[2.07014135e-04, 1.01578239e+02],
       [2.08929613e-04, 1.09851017e+02],
       [2.10862815e-04, 1.17925346e+02],
       ...,
       [9.81747943e-02, 9.98389328e+02],
       [9.90831945e-02, 9.98389328e+02],
       [1.00000000e-01, 9.98389328e+02]])

In [46]:
b = min(b_kappa_arr[:,0])
b , max(b_kappa_arr[:,0])

(0.00020701413487910414, 0.1)

There are many values of b for which the target copy number cannot be reached deterministically, hence there are many CVs which aren't physically realisable. Maximize CV and choose the corresponding $b$,$\kappa$ values:

In [47]:
kappa = b_kappa_arr[np.argmin(b_kappa_arr[:,0]),1] # take corresponding kappa for smallest b
kappa

101.57823914239846

In [59]:
utls.get_n(250, xi, gamma, beta, kappa, b, mu, delta)

1000.0065035345262

In [49]:
beta,gamma,b,kappa,mu,xi,delta

(33.120000000000005,
 0.03785142857142858,
 0.00020701413487910414,
 101.57823914239846,
 0.34657359027997264,
 0.0,
 1.0)

Find point on SS line which corresponds to $h=0.3$

In [50]:
params = {
'beta':beta,
'gamma':gamma,
'b':b,
'kappa':kappa,
'mu':mu,
'xi':xi,
'delta':delta
}

In [51]:
import mitonetworks.stoch as mts
import mitonetworks.det as mtd

In [52]:
lfc = mts.SubmitHPCSoluableFeedbackControl(ss_definition=mtd.E_linear_feedback_control_ss, nominal_params=params,hpc_jobname=None,
                                           hpc_workdir=None,c_filename=None,c_file_param_ordering_convention=None)

In [53]:
ics, h_param, ss_flag = lfc.find_ss_h_target(params)

In [54]:
ics

array([324, 376, 139, 161])

In [55]:
lfc.param_convention

['ws', 'wf', 'ms', 'mf']

In [56]:
h_param

0.29998665474793845

In [62]:
for i in params:
    print(i+' = {};'.format(params[i]))

beta = 33.120000000000005;
gamma = 0.03785142857142858;
b = 0.00020701413487910414;
kappa = 101.57823914239846;
mu = 0.34657359027997264;
xi = 0.0;
delta = 1.0;
